In [1]:
import cv2
import mediapipe as mp
import numpy as np
import re
import gensim
from glob import glob
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
folder_name = "Source"
DELIM = "\\"
re_extension = "*.mp4"
files_lst = glob(folder_name + DELIM + re_extension)

In [3]:
#files_lst

In [4]:
def get_names_list(lst, delimiter):
    return list(map(lambda x: ".".join(x.split(delimiter)[-1].split(".")[:-1]), lst))

def names_to_filenames(array: list):
    return list(map(lambda x: folder_name + DELIM + x + ".mp4", array))

In [5]:
vocabularyFilenames = get_names_list(files_lst, DELIM)
#vocabularyFilenames

In [7]:
class cv2_VideoCapture_from_list:
    
    def __init__(self, filenames: list, unnormal_names: list):
        
        self.array = filenames.copy()
        self.current_index = 0
        self.capture = cv2.VideoCapture(self.array[self.current_index])
        self.names = unnormal_names.copy()
                
    def isOpened(self):
        if self.current_index >= len(self.array):
            return False
        return self.capture.isOpened()
    
    def get(self, constant):
        if self.current_index >= len(self.array):
            return False
        return self.capture.get(constant)
    
    def set(self, constant, value):
        if self.current_index >= len(self.array):
            return False
        return self.capture.set(constant, value)
    
    def read(self):
        if self.current_index >= len(self.array):
            return False, None
        ret, frame = self.capture.read()
        if not ret:
            if self.current_index+1 < len(self.array):
                self.current_index += 1
                self.capture = cv2.VideoCapture(self.array[self.current_index])
                ret, frame = self.capture.read()
            else:
                False, None
        return ret, frame
    
    def release(self):
        self.capture.release()
        
    @property
    def name(self):
        if self.current_index >= len(self.array):
            return ""
        return self.names[self.current_index]

In [9]:
the_list = ['консультант',
 'добрый',
 'день',
 'какой',
 'операция',
 'желать',
 'клиент',
 'платить',
 'жкх',
 'консультант',
 'вы',
 'иметь',
 'карта',
 'наш',
 'банк',
 'клиент',
 'да',
 'консультант',
 'мы',
 'с',
 'вы',
 'проводить',
 'операция',
 'банкомат',
 'или',
 'личный',
 'кабинет',
 'клиент',
 'банкомат',
 'консультант',
 'хорошо',
 'пожалуйста',
 'вставить',
 'карта',
 'и',
 'вводить',
 'код']

the_unnormal = ['консультант:',
 'добрый',
 'день',
 'какую',
 'операцию',
 'желаете?',
 'клиент:',
 'оплатить',
 'жкх.',
 'консультант:',
 'у вас',
 'имеется',
 'карта',
 'нашего',
 'банка?',
 'клиент:',
 'да.',
 'консультант:',
 'мы',
 'с',
 'вами',
 'проведем',
 'операцию',
 'в банкомате',
 'или',
 'в личном',
 'кабинете?',
 'клиент:',
 'В банкомате.',
 'консультант:',
 'хорошо,',
 'пожалуйста',
 'вставьте',
 'карту',
 'и',
 'введите',
 'пин-код.']

def PutText(frame, word):
    font                   = cv2.FONT_HERSHEY_COMPLEX
    bottomLeftCornerOfText = (70,265)
    fontScale              = 1
    fontColor              = (255,0,0)
    thickness              = 1
    lineType               = 1
    cv2.putText(frame,word, bottomLeftCornerOfText, font, fontScale,fontColor,thickness,lineType)

background = cv2.imread( 'background.jpg' , cv2.IMREAD_UNCHANGED)
cap = cv2_VideoCapture_from_list(names_to_filenames(the_list), the_unnormal)

writer = cv2.VideoWriter("surdoperevod.mp4", cv2.VideoWriter_fourcc(*'H264'), 25, (320,280))

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        #cap.set(cv2.CAP_PROP_FPS, FPS)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=4, circle_radius=2) 
                                 )               

        comb = np.concatenate((image,background),axis=0)
        PutText(comb, cap.name)
        writer.write(comb)
        cv2.imshow('Deaf Avatar', comb)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    writer.release()
    cv2.destroyAllWindows()